In [0]:
import scandir
import os
import rpy2
from rpy2.robjects import pandas2ri
pandas2ri.activate()
import rpy2.robjects as ro
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
import dill
import random
import vcf
from hdfstorehelper import HDFStoreHelper
import statsmodels.api as sm
import statsmodels.formula.api as smf
import operator
import traceback
%load_ext rpy2.ipython
from rpy2.robjects import pandas2ri as p2r
p2r.activate()
r = ro.r
import shutil
from utils import read_df, save_df
from pathlib import Path, PurePath
from ipyparallel import Client
from collections import Counter, defaultdict, namedtuple

In [0]:
analysis_dir_notimp = "/home/cfriedline/eckertlab/gypsy_indiv/raw_demult/analysis/samtools1.3_masurca3/ni/"
analysis_dir_imp = "/home/cfriedline/eckertlab/gypsy_indiv/raw_demult/analysis/samtools1.3_masurca3/beagle40/"

In [0]:
analysis_dir = [analysis_dir_notimp, analysis_dir_imp]

In [0]:
gps = {'QC32':[47.2509807, -79.4060515],
      'QC93': [46.9089631, -70.8061075],
      'NC': [36.449125, -76.024672],
      'NY': [42.897768, -74.094761],
      'VA1': [38.657615, -77.463603],
      'VA2': [38.857470, -77.695003]}
gps_df = pd.DataFrame(gps).T
gps_df.columns = ['lat','lon']

latlon = pandas2ri.py2ri(gps_df[['lon', 'lat']])

In [0]:
%%R
library(sp)
library(raster)

In [0]:
raster = r("raster")
extract = r("extract")

In [0]:
bioclim_dir = "/home/cfriedline/eckertlab/bioclim"
bioclim = !ls {bioclim_dir}/*.bil
bioclim = sorted(bioclim)
bioclim_df = pd.DataFrame(gps_df)
for b in bioclim:
    rast = raster(b)
    bio = os.path.basename(b).replace(".bil", "").replace("_", "").upper()
    vals = pd.DataFrame(pandas2ri.ri2py(extract(rast, latlon)))
    vals.index = bioclim_df.index
    vals.columns = [bio]
    bioclim_df = bioclim_df.join(vals)
bioclim_df = bioclim_df.sort_index()

In [0]:
for d in analysis_dir:
    save_df(d, "bioclim_df", bioclim_df)

In [0]:
region_id = 1
for popname in bioclim_df.index:
    bioclim_df.ix[popname, 'region'] = region_id
    region_id+=1

In [0]:
bioclim_env_files = []
for bio in bioclim_df[[x for x in bioclim_df if 'BIO' in x]]:
    bio_temp = bioclim_df[[bio, 'region']]
    bio_temp.index.name = "CLST"
    bio_temp.columns = ["ENV", "REG"]
    bio_out = os.path.join(analysis_dir[0], "../%s.txt" % bio)
    bioclim_env_files.append(bio_out)
    bio_temp.to_csv(bio_out, sep="\t", header=True, index=True)

In [0]:
z12_swapped = [read_df(x, 'z12_swapped') for x in analysis_dir]
z12 = [read_df(x, 'z12_df') for x in analysis_dir]


In [0]:
def get_correction(n):
    #for finite sample size
    return (2*n)/(2*n-1)

def get_allele_freqs(locus, debug):
    import pandas as pd
    try:
        c = locus[locus != -1].value_counts()
        if not len(c):
            return pd.Series()
        total_alleles = 2.0*sum(c)    
        num_individuals = sum(c)
        P = 0
        Q = 0
        PQ = 0
        if 0 in c:
            P = 2*c[0]
        if 2 in c:
            Q = 2*c[2]
        if 1 in c:
            PQ = c[1]
        P += PQ
        Q += PQ
        p = P/total_alleles
        q = Q/total_alleles
        assert p + q == 1.0
        He = 2 * p * q * get_correction(num_individuals)
        Ho = PQ*1.0/num_individuals
        Fis = 1 - (Ho/He)
        #print p, q, He, Ho, Fis


        ret = pd.Series({"p":p, 
                          "q":q,
                          "P":P,
                          "Q":Q,
                          "He":He,
                          "Ho":Ho, 
                          "Fis":Fis})
        if debug:
            print(ret)
        return ret
    except:
        print(c)

In [0]:
allele_freqs = [x.ix[:,:-2].apply(get_allele_freqs, args=(False,)) for x in z12_swapped]

In [0]:
allele_freqs[0].shape, allele_freqs[1].shape

In [0]:
for i, af in enumerate(allele_freqs):
    save_df(analysis_dir[i], 'allele_freqs_squat', af)    

In [0]:
gt_base_df = [read_df(x, 'gt_base_df') for x in analysis_dir]

In [0]:
df = gt_base_df[1]

In [0]:
pops = sorted(set([x.split("_")[0] for x in df.index]))

In [0]:
def count_genotypes(snp):
    counts = Counter()
    for gt in snp:
        try:
            float(gt) #if gt is nan
        except:
            counts[gt[0]]+=1
            counts[gt[-1]]+=1
    return sorted(counts.items(), key=operator.itemgetter(1))

In [0]:
def add_af_to_count(data):
    total = sum([x[1] for x in data])
    ret = []
    for elem in data:
        elem = list(elem)[0:2]
        elem.append(elem[1]/total)
        ret.append(elem)
    if len(ret) == 1:
        ret.append()
    return ret

In [0]:
gt_counts = df.apply(count_genotypes)

In [0]:
SNP = namedtuple("SNP", ['name',
                             'minor_allele',
                             'minor_count',
                             'minor_freq',
                             'major_allele',
                             'major_count',
                             'major_freq',
                             'contig',
                             'loc',
                             'pop_data'])

In [0]:
def get_allele_data(count_data):
    allele_data = {}
    for snp, data in count_data.to_dict().items():
        name_data = snp.split("_")
        if len(data) == 2:
            total = np.sum([x[1] for x in data])
            allele_data[snp] = SNP(snp, data[0][0],
                                              data[0][1],
                                              data[0][1]/total,
                                              data[1][0],
                                              data[1][1],
                                              data[1][1]/total,
                                              "_".join(name_data[0:-1]),
                                              int(name_data[-1]),
                                              None)
        else:
            total = data[0][1]
            allele_data[snp] = SNP(snp, None,
                                              None,
                                                None,
                                              data[0][0],
                                              data[0][1],
                                              data[0][1]/total,
                                              "_".join(name_data[0:-1]),
                                              int(name_data[-1]),
                                              None)
    return allele_data

In [0]:
allele_data = get_allele_data(gt_counts)

In [0]:
df['population'] = df.apply(lambda x: x.name.split("_")[0], axis=1)

In [0]:
data.head()

In [0]:
pop_allele_data = {}

def add_allele_freq(gt_list):
    data = gt_list
    if len(gt_list) == 2:
        total = data[0][1]+data[1][1]
        return {data[0][0]:[data[0][1], data[0][1]/total],
               data[1][0]:[data[1][1], data[1][1]/total]}
    else:
        return {data[0][0]:[data[0][1], 1.0]}

for group, data in df.groupby('population'):
    data = data.drop('population', axis=1)
    print(group, data.shape)
    gt = data.apply(count_genotypes).apply(add_allele_freq)
    pop_allele_data[group] = gt.to_dict()

In [0]:
allele_data['ctg7180006068889_3947']

In [0]:
pop_allele_data['NC']['ctg7180005818477_209']

In [0]:
pimass_gt = [read_df(x, '_pimass_gt').replace("NA", np.nan) for x in analysis_dir]

In [0]:
pimass_gt[0].head()

In [0]:
pimass_gt[1].head()

In [0]:
gt_ref_alt = [read_df(x, 'gt_ref_alt_df') for x in analysis_dir]

In [0]:
gt_ref_alt_minor_major = [read_df(x, 'gt_ref_alt_minor_major') for x in analysis_dir]

In [0]:
gt_ref_alt[0].head()

In [0]:
translation_df = pd.read_csv("translation_table.csv", sep="\t", index_col=0)
def get_translated_name(n):
    n = n.strip()
    if n in translation_df.index:
        row = translation_df.ix[n.strip()]
        return "%s_%d_%d" % (row['pop'], row.indiv, row.dup)
    return n

In [0]:
for p in pimass_gt:
    p.columns = [get_translated_name(x) for x in p.columns]

In [0]:
pimass_pheno = [read_df(x, '_pimass_pheno') for x in analysis_dir]

In [0]:
pimass_pheno[1].head()

In [0]:
for pp in pimass_pheno:
    pp.columns = [x.lower().replace(" ", "_") for x in pp.columns]

In [0]:
phenos = [x[['mass_resid', 'pupual_duration_resid', 'total_dev_time_resid']] for x in pimass_pheno]
#phenos = [x[['massx', 'pdx', 'tdtx']] for x in pimass_pheno]

In [0]:
rc = Client(profile="sge")
dview = rc[:]
lview = rc.load_balanced_view()
len(dview)

In [0]:
@lview.remote()
def calc_alpha(args):
    pheno, data = args
    res = {}
    import statsmodels.api as sm  
    for snp in data.columns[3:]:
        model = sm.OLS(data[pheno], data[snp], missing='drop')
        fit = model.fit()
        res[snp] = fit.params.values[0]
    return res

In [0]:
alphas = []
for i, pheno_df in enumerate(phenos):
    alphas.append({})
    data = pheno_df.join(pimass_gt[i].T.ix[2:,])
    data.columns = [x.replace(" ", "_") for x in data.columns]
    data = data.astype(float)
    for pheno in data.columns[0:3]:
        alphas[i][pheno] = calc_alpha((pheno, data))
        print(i, pheno)
#         for snp in data.columns[3:]:
#             alphas[i][pheno][snp] = calc_alpha(data)
#             model = sm.OLS(data[pheno], data[snp], missing='drop')
#             fit = model.fit()
#             alphas[i][pheno][snp] = fit.params.values[0]

In [0]:
for i, alpha in enumerate(alphas):
    for pheno in alpha:
        print(i, pheno, alpha[pheno].ready())

In [0]:
for i, alpha in enumerate(alphas):
    for pheno in alpha:
        alphas[i][pheno] = alpha[pheno].r

In [0]:
pimass_pheno[1].head()

In [0]:
pimass_gt[1].head()

In [0]:
alpha_df = [pd.DataFrame(x) for x in alphas]

In [0]:
alpha_df[1].head()

In [0]:
for i, ad in enumerate(alpha_df):
    save_df(analysis_dir[i], 'alpha_df', ad)
#     hdfs[i]['alpha_df'] = ad

In [0]:
pheno_names = phenos[0].columns

In [0]:
pheno_names

In [0]:
def get_sig_pheno_name(col):
    return {'mass_resid': 'mass',
           'pupual_duration_resid': 'pd',
           'total_dev_time_resid': 'tdt',
           'massx': 'mass',
           'tdtx': 'tdt',
           'pdx':'pd'}[col]

In [0]:
def assign_a1_a2(snp):    
    if snp.alpha > 0:
        return (snp.minor_allele, snp.major_allele, snp.minor_freq, "minor")
    else:
        return (snp.major_allele, snp.minor_allele, snp.major_freq, "major")

In [0]:
%load_ext cython

In [0]:
def get_A1_pop_freq(snp, pop):
    allele = snp['A1']
    if allele in paf[pop][snp.name]:
        return paf[pop][snp.name][allele][-1]
    else:
        return 0.0

def write_freqs_file(df, paf, outdir, pheno, imputed, key, sig_key):
    df = df.copy()
    outfile = os.path.join(outdir, "freqs.file.%s.%s.%s.%s.txt" % (sig_key, key, pheno, imputed))
    df.index.name = "SNP"
    dfs = []
    for pop in paf:
        df2 = df.copy()
        df2['A1_pop_freq'] = df2.apply(get_A1_pop_freq, args=(pop,), axis=1)
        df2['CLST'] = pop
        df2 = df2[['CLST', 'A1', 'A2', 'A1_pop_freq', 'loc', 'contig']]
        df2.columns = ['CLST', 'A1', 'A2', 'FRQ', 'POS', 'CHR']
        dfs.append(df2)
    combined = pd.concat(dfs)
    combined.to_csv(outfile, header=True, index=True, sep="\t")
    return outfile, combined['CLST'].unique()

def write_gwas_data_file(df, outdir, pheno, imputed, sig_key):
    df = df.copy()
    outfile = os.path.join(outdir, "gwas.data.file.%s.%s.%s.txt" % (sig_key, pheno, imputed))
    df.index.name = "SNP"
    df = df[['A1', 'A2', 'alpha', 'A1_freq']]
    df.columns = ["A1", "A2", "EFF", "FRQ"]
    df.to_csv(outfile, header=True, index=True, sep="\t")
    return outfile
    

def write_env_var_data_files(paf, pops, outdir, imputed, pheno, sig_key):
    outfiles = []
    outfile = os.path.join(outdir, "env.var.data.file.%s.%s.%s.txt" % (sig_key, pheno, imputed))
    outfiles.append(outfile)
    with open(outfile, "w") as o:
        o.write("CLST\tENV\tREG\n")
        pop_id = 1
        for pop in sorted(paf):
            if pop in pops:
                o.write("%s\t%g\t%d\n" % (pop, np.random.randn(), pop_id))
            pop_id += 1
    return outfiles
  

def write_match_pop_file(df, paf, outdir, imputed, matchpop, pheno, sig_key):
    outfile = os.path.join(outdir, "match.pop.file.%s.%s.%s.%s.txt" % (sig_key, pheno, matchpop, imputed))
    df2 = df.copy()
    df2.index.name="SNP"
    df2['CLST'] = matchpop
    df2['A1_freq'] = df2.apply(get_A1_pop_freq, args=(matchpop,), axis=1)
    df2 = df2[['CLST', 'A1', 'A2', 'A1_freq', 'loc', 'contig']]
    df2.columns = ['CLST', 'A1', 'A2', 'FRQ', 'POS', 'CHR']
    df2.to_csv(outfile, header=True, index=True, sep="\t")
    return outfile

def get_squat_vars(pheno,
                   gwas_data_file, 
                  gwas_freqs_file, 
                  env_var_files,
                  match_pop_file,
                  full_freqs_file,
                  num_snps):
    d = {"gwas.data.file": "'%s'" % gwas_data_file,
         "freqs.file": "'%s'" % gwas_freqs_file,
         "env.var.data.files": "list(%s)" % ','.join(["'%s'" % x for x in env_var_files]),
         "match.pop.file": "'%s'" % match_pop_file,
         "full.dataset.file": "'%s'" % full_freqs_file,
         "path":"'%s'" % pheno,
         "match.categories":"c('FRQ')",
         "match.bins":"list(seq(0,0.5,0.02))",
         "cov.SNPs.per.cycle":20000,
         "cov.cycles":100,
         "null.phenos.per.cycle":10000,
         "null.cycles":10,
         "load.cov.mat":"F",
         "sim.null":"T",
         "check.allele.orientation":"T"}
    return ',\n'.join("%s=%s" % (key,val) for (key,val) in list(d.items()))

def create_squat_run_file(pheno, outdir, squat_vars, sig_key):
    squat_dir = "/home/cfriedline/eckertlab/src/PolygenicAdaptationCode/Scripts/"
    out_dir = os.path.join(outdir, "squat_%s" % sig_key)
    res_dir = os.path.join(out_dir, pheno)
    if not os.path.exists(out_dir):
        os.mkdir(out_dir)
        os.symlink(squat_dir, os.path.join(out_dir, "Scripts"))
    squat_file = os.path.join(out_dir, "squat_%s.%s.r" % (sig_key, pheno))
    with open(squat_file, "w") as o:
        o.write("system('rm -rf %s')\n" % res_dir)
        o.write("setwd('/home/cfriedline/eckertlab/src/PolygenicAdaptationCode')\n")
        o.write("source('%s')\n" % os.path.join(squat_dir, "CreateTraitFile.R"))
        o.write("source('%s')\n" % os.path.join(squat_dir, "functions.R"))
        o.write("setwd('%s')\n" % out_dir)
        o.write("PolygenicAdaptationFunction(%s)\n" % squat_vars)
    return squat_file



In [0]:
allele_df = pd.DataFrame(allele_data, index=SNP._fields).T

In [0]:
import datetime
def current_time():
    return str(datetime.datetime.now().time())

In [0]:
monomorphic_pop_snps = set()
for pop in pop_allele_data:
    d = pop_allele_data[pop]
    for snp in d:
        if len(d[snp]) == 1:
            monomorphic_pop_snps.add(snp)
len(monomorphic_pop_snps)

In [0]:
full.ix[sig.rs]

In [0]:
sig

In [0]:
'ctg7180005879129_2189' in monomorphic_pop_snps

In [0]:
# create squat files for range snps
range_squat_files = []
from collections import defaultdict
squat_dict = defaultdict()
for i, d in enumerate(analysis_dir):
    squat_dict[d] = defaultdict()
    if i == 0:
        continue
    outdir = analysis_dir[i]
    for name in pheno_names:
        squat_dict[d][name] = defaultdict()
        paf = pop_allele_data
        pheno_hdf = get_sig_pheno_name(name)
        snp_ranges = read_df(d, '%s_quant_range' % pheno_hdf)
        full = read_df(d, 'mcmc_%s_hmean' % pheno_hdf)
        full.index = full.rs
        full.index.name="%s_%d" % (name, i)
        full = full.drop(monomorphic_pop_snps, errors="ignore")
        alpha = alphas[i][name]
        full['alpha'] = full.apply(lambda x: alpha[x.name], axis=1)       
        full = full.drop("rs", axis=1)
        full = full.join(allele_df, how="inner")
        full['allele_data'] = full.apply(assign_a1_a2, axis=1)
        full[['A1','A2','A1_freq','effect_allele']] = full['allele_data'].apply(pd.Series)
        print(current_time(), name, len(full))
        try:
            for j, val in enumerate(snp_ranges["0"].values):
                exp_dir = os.path.join(outdir, "squat_ranges")
                exp_dir = os.path.join(exp_dir, "%s_range_%d" % (pheno_hdf, j))
                if not os.path.exists(exp_dir):
                    os.makedirs(exp_dir)
                else:
                    shutil.rmtree(exp_dir)
                    os.makedirs(exp_dir)
                sig = read_df(d, 'range_snps_%s_%d' % (pheno_hdf, j))
                sig.index = sig.rs
                sig = sig.drop(monomorphic_pop_snps, errors="ignore")
                sig_key = "range_%d" % j
                print(current_time(), sig_key)
                gwas_data_file = write_gwas_data_file(full.ix[sig.rs], exp_dir, name, i, sig_key)
                gwas_freqs_file, gwas_pops = write_freqs_file(full.ix[sig.rs], paf, exp_dir, name, i, "gwas", sig_key)
                match_pop_file = write_match_pop_file(full, paf, exp_dir, i, "QC32", name, sig_key)
                full_freqs_file, full_pops = write_freqs_file(full, paf, exp_dir, name, i, "full", sig_key)
                squat_vars = get_squat_vars(name, gwas_data_file,
                              gwas_freqs_file, 
                              bioclim_env_files,
                              match_pop_file,
                              full_freqs_file, len(full))
                squat_file = create_squat_run_file(name, exp_dir, squat_vars, sig_key)
                print(current_time(), "wrote %s" % squat_file)
                range_squat_files.append(squat_file)
                
                squat_dict[d][name][val] = {'gwas_data_file': gwas_data_file,
                                           'gwas_freqs_file': gwas_freqs_file,
                                           'gwas_pops': gwas_pops,
                                           'match_pop_file': match_pop_file,
                                           'full_freqs_file': full_freqs_file,
                                           'full_pops': full_pops,
                                           'squat_file': squat_file}
                #break
        except:
            traceback.print_exc()
        
        #break
    
    #break

In [0]:
ni = open("/home/cfriedline/eckertlab/gypsy_indiv/raw_demult/analysis/samtools1.3_masurca3/ni/squat_parallel.txt", "w")
bg = open("/home/cfriedline/eckertlab/gypsy_indiv/raw_demult/analysis/samtools1.3_masurca3/beagle40/squat_parallel.txt", "w")
ni_files = []
bg_files = []
for rsf in range_squat_files:
    o = ni
    if 'beagle40' in rsf:
        o = bg
        bg_files.append(rsf)
    else:
        ni_files.append(rsf)
    o.write("cd %s && R --slave --vanilla < %s > %s.log\n" % (os.path.dirname(rsf), rsf, rsf))
[x.close() for x in [ni, bg]];

```bash
~/bin/parallel --no-notice --bar -a squat_parallel.txt
```

In [0]:
range_outputs = !find {os.path.commonprefix(range_squat_files)} | grep 'Output$'

In [0]:
range_outputs = !find {os.path.commonprefix(bg_files)} | grep 'Output$'

In [0]:
len(range_outputs)

In [0]:
range_output_data = []
used_outputs = []
for output in range_outputs:
    p = PurePath(output)
    d = p.parents[4]
    pheno_range = os.path.basename(os.path.dirname(os.path.dirname(os.path.dirname(output))))
    pheno_data = pheno_range.split("_")
    pheno = pheno_data[0]
    range_id = int(pheno_data[-1])
    ranges = read_df(str(d), '%s_quant_range' % pheno)
    r("rm(list=ls())")
    for obj in os.listdir(output):
        obj = os.path.join(output, obj)
        r("load('%s')" % obj)
    try:
        range_output_data.append({'pheno': pheno,
                                'pheno_range': pheno_range,
                                      'cutoff':ranges.ix[range_id,"0"],
                                          'Qx': r('the.stats$Qx')[0],
                                          'Pr(Qx)': r('p.vals$Qx')[0],
                                          'Fst': r('the.stats$Fst.comp')[0],
                                          'Pr(Fst)': r('p.vals$Fst.comp')[0],
                                          'LD': r('the.stats$LD.comp')[0],
                                          'Pr(LD)': r('p.vals$LD.comp')[0]})
        used_outputs.append(output)
    except:
        pass

In [0]:
%%R
ls()

In [0]:
num_gwas = {}
for uo in used_outputs:
    res = !wc -l {os.path.dirname(os.path.dirname(os.path.dirname(uo)))}/gwas*.txt
    res = res[0].split()
    n = res[0]
    p = os.path.basename(os.path.dirname(res[1]))
    num_gwas[p] = n

In [0]:
#range_output_data = []
for output in range_outputs:
    p = PurePath(output)
    d = p.parents[4]
    pheno_range = os.path.basename(os.path.dirname(os.path.dirname(os.path.dirname(output))))
    pheno_data = pheno_range.split("_")
    pheno = pheno_data[0]
    range_id = int(pheno_data[-1])
    if range_id ==  9:
        ranges = read_df(str(d), '%s_quant_range' % pheno)
        r("rm(list=ls())")
        for obj in os.listdir(output):
            obj = os.path.join(output, obj)
            print(obj)
            r("load('%s')" % obj)
        break

In [0]:
%%R
hist(null.stats$Qx, xlim=c(0, the.stats$Qx+50))
abline(v=the.stats$Qx, col="red")
text(150, 1000, "Qx", col="red")
hist(null.stats$LD.component, xlim=c(-5, the.stats$LD.component+50))
abline(v=the.stats$LD.component, col="red")
text(150, 1000, "LD", col="red")

In [0]:
range_output_df = pd.DataFrame(range_output_data)

In [0]:
range_output_df.head()

In [0]:
range_output_df['num'] = range_output_df.apply(lambda x: num_gwas[x.pheno_range], axis=1)

In [0]:
range_output_df.head()

In [0]:
save_df(analysis_dir[1], "range_output_df", range_output_df)

In [0]:
import matplotlib.patches as mpatches
sns.set_context("talk")
for group, data in range_output_df.groupby("pheno"):
    data = data.sort_values("cutoff", ascending=False)
    print(len(data))
    fig = plt.figure()
    fig.set_size_inches(30,5)
    ax1 = fig.add_subplot(131)
    ax1.plot(data.cutoff, data.Qx, color="blue")
    plt.ylabel("Qx")
    plt.xlabel("PIP cutoff")
    ax2 = ax1.twinx()
    ax2.plot(data.cutoff, data['Pr(Qx)'], color="red")
    plt.ylabel("Pr(Qx)")
    ax2.grid(None)
    plt.title(group)
    red_patch = mpatches.Patch(color='red', label='Pr(Qx)')
    blue_patch = mpatches.Patch(color='blue', label='Qx')
    for row in data.index:
        ax1.annotate(data.ix[row,'num'], 
                     (data.ix[row, 'cutoff'], data.ix[row, 'Qx']),
                    xytext = (-1, 1),
                    textcoords = 'offset points',
                    ha = 'right', va = 'bottom',
                    bbox = dict(boxstyle = 'round,pad=0.5', fc = 'yellow', alpha = 0.5))
    
    plt.legend(handles=[blue_patch, red_patch], loc="upper center")
    
    ax3 = plt.subplot(132)
    ax3.plot(data.cutoff, data.LD, color="blue")
    plt.ylabel("LD")
    ax4 = ax3.twinx()
    ax4.plot(data.cutoff, data['Pr(LD)'], color="red")
    plt.ylabel("Pr(LD)")
    ax4.grid(None)
    red_patch = mpatches.Patch(color='red', label='Pr(LD)')
    blue_patch = mpatches.Patch(color='blue', label='LD')
    plt.legend(handles=[blue_patch, red_patch], loc="upper center")
    plt.title(group)
    plt.subplots_adjust(wspace=0.3)
    
    ax5 = plt.subplot(133)
    ax5.plot(data.cutoff, data.Fst, color="blue")
    plt.ylabel("Fst")
    ax6 = ax5.twinx()
    ax6.plot(data.cutoff, data['Pr(Fst)'], color="red")
    plt.ylabel("Pr(Fst)")
    ax6.grid(None)
    red_patch = mpatches.Patch(color='red', label='Pr(Fst)')
    blue_patch = mpatches.Patch(color='blue', label='Fst')
    plt.legend(handles=[blue_patch, red_patch], loc="upper center")
    plt.title(group)
    plt.subplots_adjust(wspace=0.3)
    
    
    plt.show()

In [0]:
squat_files

In [0]:
%%R
setwd("~/")

In [0]:
for squat_file in squat_files:
    print(squat_file)
    r("source('%s')" % squat_file)

In [0]:
with open(os.path.join(os.path.dirname(os.path.dirname(analysis_dir_imp)), "squat_results.txt"), "w") as o:  
    o.write("%s\n" % "\t".join(["significance", "phenotype", "imputation", "Qx", "Pr(Qx)", 
                               "Fst", "Pr(Fst)", "LD", "Pr(LD)"]))
    for squat_file in squat_files:
        squat_dir = os.path.dirname(squat_file)
        datatype = os.path.basename(os.path.dirname(squat_dir))
        pheno = os.path.basename(squat_file).split(".")[1]
        pheno_dir = os.path.join(squat_dir, pheno)
        output_dir = os.path.join(pheno_dir, "Output")
        assert os.path.exists(output_dir)

        for obj in os.listdir(output_dir):
            obj = os.path.join(output_dir, obj)
            r("load('%s')" % obj)
            
        res = "\t".join([str(x) for x in [os.path.basename(squat_dir),
                                          pheno,
                                          datatype,
                                          r('the.stats$Qx')[0],
                                          r('p.vals$Qx')[0],
                                          r('the.stats$Fst.comp')[0],
                                          r('p.vals$Fst.comp')[0],
                                          r('the.stats$LD.comp')[0],
                                          r('p.vals$LD.comp')[0]]])
        print(res)
        o.write("%s\n" % res)
    